# Testing

### Environment Setting

In [1]:
init_mu = 25
init_sigma = init_mu / 3
init_beta = init_sigma / 2  # skill chain length
init_tau = init_sigma / 100  # dynamic factor

### Loading Data

In [2]:
def print_file_head(file_path, col_names, rows=100):
    f = open(file_path, encoding='utf8')
    for t, row in enumerate(DictReader(f, fieldnames=col_names, delimiter=',')):
        print(row)

        if t > rows - 2:
            break
    f.close()

In [8]:
from csv import DictReader

user_file_path = r'C:\Users\dongwan.kim\Desktop\da_work\T5\20170628_T5_MatchData\20170628_T5_char_ids1.csv'
user_file_header = ['uuid', 'name', 'reg_date']

match_file_path = r'C:\Users\dongwan.kim\Desktop\da_work\T5\20170628_T5_MatchData\20170628_T5_MatchData1_3.csv'
match_file_header = ['tddate', 'uuid', 'game_srl', 'win', 'min_reg_date', 'rk_team_members', 'rk_match', 'num_team_users', 'win_team_num', 'lose_team_num']

match_quality_out_file_path = r'C:\Users\dongwan.kim\Desktop\da_work\T5\20170628_T5_MatchData\match_quality_out_file_10only.csv'

# print_file_head(match_file_path, user_file_header, 10)
print_file_head(match_file_path, match_file_header, 2)

{'win_team_num': '5', 'lose_team_num': '3', 'tddate': '\ufeff2017-06-15', 'num_team_users': '8', 'min_reg_date': '2017-06-15 11:11:46.000', 'rk_match': '1', 'game_srl': 'ec2-52-78-245-65.ap-northeast-2.compute.amazonaws.com_4_5941ebb5', 'uuid': '26', 'rk_team_members': '6', 'win': '0'}
{'win_team_num': '5', 'lose_team_num': '3', 'tddate': '2017-06-15', 'num_team_users': '8', 'min_reg_date': '2017-06-15 11:11:46.000', 'rk_match': '1', 'game_srl': 'ec2-52-78-245-65.ap-northeast-2.compute.amazonaws.com_4_5941ebb5', 'uuid': '29', 'rk_team_members': '7', 'win': '1'}


### TrueSkill Simulation

In [9]:
def update_rate_and_get_match_quality(match_composition):
    # print('team:', match_composition)
    
    win_team_ratings = {}
    for member in match_composition['win_team_members']:
        users_mu_sigma = user_ratings[member]  # ur is tuple composed of (mu, sigma) representing use's rating
        users_rating = env.Rating(mu=users_mu_sigma[0], sigma=users_mu_sigma[1])
        win_team_ratings[member] = users_rating
    
    lose_team_ratings = {}
    for member in match_composition['lose_team_members']:
        users_mu_sigma = user_ratings[member]  # ur is tuple composed of (mu, sigma) representing use's rating
        users_rating = env.Rating(mu=users_mu_sigma[0], sigma=users_mu_sigma[1])
        lose_team_ratings[member] = users_rating

    match_quality = env.quality(rating_groups=[win_team_ratings, lose_team_ratings])  # calculating match quality before the match
    # print('match:', match_composition['game_srl'], 'quality:', match_quality)
    
    new_rate = env.rate([win_team_ratings, lose_team_ratings])  # calculating new rates after the match
    
    # updating rate for winers
    for key, value in new_rate[0].items():
        # print('key:', key, 'value:', value)
        user_ratings[key] = (value.mu, value.sigma)  # update rate for each user
    
    # updating rate for loses
    for key, value in new_rate[1].items():
        # print('key:', key, 'value:', value)
        user_ratings[key] = (value.mu, value.sigma)  # update rate for each user
            
    return (match_composition['game_srl'], str(match_quality))

In [11]:
import trueskill as ts

# init user ratings
user_ratings = {}
f = open(user_file_path, encoding='utf8')
for t, row in enumerate(DictReader(f, fieldnames=user_file_header, delimiter=',')):
    user_ratings[row['uuid']] = (init_mu, init_sigma)
f.close()

rk_team_last = None
match_composition = {}
match_composition['win_team_members'] = []
match_composition['lose_team_members'] = []
match_composition['rk_match'] = None
match_composition['game_srl'] = None

env = ts.TrueSkill(mu=init_mu, sigma=init_sigma, beta=init_beta, tau=init_tau, draw_probability=0, backend='mpmath')
r_file = open(match_file_path, encoding='utf8')
w_file = open(match_quality_out_file_path, encoding='utf8', mode='w')
for t, row in enumerate(DictReader(r_file, fieldnames=match_file_header, delimiter=',')):
    if row['num_team_users'] == '10' and row['win_team_num'] == row['lose_team_num'] and int(row['win_team_num']) > 0 and int(row['lose_team_num']) > 0:  # nop fair matches
        
        if rk_team_last == row['rk_match'] or rk_team_last is None:
            match_composition['rk_match'] = row['rk_match']
            match_composition['game_srl'] = row['game_srl']
            if row['win'] == '1':
                # work for win team
                match_composition['win_team_members'].append(row['uuid'])
                
            else:
                # work for lose team
                match_composition['lose_team_members'].append(row['uuid'])
            rk_team_last = row['rk_match']
        else:
                       
            # work for each match with match_composition
            try:
                mq = update_rate_and_get_match_quality(match_composition)
            except:
                print(match_composition)
                
            w_file.write(','.join(mq) + '\n')
            
            
            # initialize for the next 
            match_composition = {}
            match_composition['win_team_members'] = []
            match_composition['lose_team_members'] = []

            rk_team_last = row['rk_match']
            if row['win'] == '1':
                # work for win team
                match_composition['win_team_members'].append(row['uuid'])
            else:
                # work for lose team
                match_composition['lose_team_members'].append(row['uuid'])
    
    #if int(row['rk_match']) > 6000:
    #    break
r_file.close()
w_file.close()